In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import torch

import tsvar

In [3]:
mu = torch.tensor([0.2, 0.1])
beta = torch.tensor([1.1, 1.5])
A = torch.tensor([
    [0.7, 0.3],
    [0.1, 0.9]
])
dim = 2

#coeffs = torch.cat((mu, beta[0], A.flatten()))

In [97]:
end_time = 100000.0

wold_sim = tsvar.simulate.GrangeBuscaSimulator(mu_rates=mu, Alpha_ba=A, Beta_b=beta)
events = wold_sim.simulate(end_time)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]

print(f"Simulated {sum(map(len, events))} events with end time: {end_time}")
print(list(map(len, events)))

Simulated 102680 events with end time: 100000.0
[50859, 51821]


In [98]:
class WoldModelSimple(tsvar.wold_model.WoldModel):
    
    def set_data(self, events, end_time=None):
        super().set_data(events, end_time)
        self.n_params = self.dim * (self.dim - 1) + self.dim

    def log_likelihood(self, coeffs):
        # Extract each set of parameters
        mu = coeffs[:self.dim]
        beta = coeffs[self.dim:2*self.dim]
        alpha = coeffs[2*self.dim:].reshape(self.dim, self.dim)
        # Compute the log-likelihood
        log_like = 0
        for i in range(self.dim):
            # Compute the intensity at each event
            lam_ik_arr = mu[i] + torch.sum(
                self.valid_mask_ikj[i] * alpha[:,i] /(beta.unsqueeze(0) + self.delta_ikj[i]), 
                axis=1)
            # Add the log-intensity term
            log_like += lam_ik_arr[:-1].log().sum()
            # Subtract the integral term
            log_like -= lam_ik_arr[0] * self.events[i][0]
            log_like -= torch.sum(lam_ik_arr[1:] * (self.events[i][1:] - self.events[i][:-1]))
        return log_like

In [99]:
model = tsvar.wold_model.WoldModel()
model.set_data(events, end_time)

coeffs_true = torch.cat((mu, beta - 1, A.flatten()))

x0 = torch.cat((
    0.01 * torch.ones(dim, dtype=torch.float),
    1.0 * torch.ones(dim, dtype=torch.float),
    0.5 * torch.ones((dim, dim), dtype=torch.float).flatten()
))

C = 1000.0 * torch.ones(len(coeffs_true))
prior = tsvar.priors.GaussianPrior(C=C)

optimizer = torch.optim.Adam([x0], lr=0.5)

learner = tsvar.learners.MLELearner(
    model=model, 
    prior=prior, 
    optimizer=optimizer,
    tol=1e-5,
    max_iter=100000,
    debug=False)

learner_callback = tsvar.utils.callbacks.LearnerCallbackMLE(x0, coeffs_true.numpy(), print_every=10)

In [100]:
coeffs_true

tensor([0.2000, 0.1000, 0.1000, 0.5000, 0.7000, 0.3000, 0.1000, 0.9000])

In [101]:
x0

tensor([0.0100, 0.0100, 1.0000, 1.0000, 0.5000, 0.5000, 0.5000, 0.5000])

In [102]:
coeffs_hat = learner.fit(events, end_time, x0, callback=learner_callback)

iter:   400 | dx: 1.3173e-05 | loss: 1.6653e+05    loss = 166531.28125 Converged!


In [103]:
coeffs_hat.detach().numpy().round(2)

array([0.21, 0.1 , 0.08, 0.26, 0.69, 0.31, 0.07, 0.78], dtype=float32)

In [104]:
coeffs_true.detach().numpy().round(2)

array([0.2, 0.1, 0.1, 0.5, 0.7, 0.3, 0.1, 0.9], dtype=float32)

In [105]:
-model.log_likelihood(x0)

tensor(181507.1562)

In [106]:
-model.log_likelihood(coeffs_hat)

tensor(166531.2812, grad_fn=<NegBackward>)

In [107]:
-model.log_likelihood(coeffs_true)

tensor(166539.1875)